# Лабораторная работа №3
## Выполнила студентка группы БФИ2001 Калмыкова Дарья Сергеевна

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def prefix(text):
    n = len(text)
    pi = [0] * n
    for i in range(1, n):
        j = pi[i-1]
        while j > 0 and text[j] != text[i]:
            j = pi[j-1]
        if text[i] == text[j]:
            j += 1
        pi[i] = j
    return pi

def kmp_search(text, substring, start_index=0):
    j = 0
    pi = prefix(substring)
    for i in range(start_index, len(text)):
        while j > 0 and text[i] != substring[j]:
            j = pi[j-1]
        if text[i] == substring[j]:
            j += 1
        if j >= len(substring):
            return i - j + 1
    return None

text = input('Введите текст: ')
substring = input('Введите подстроку для поиска: ')
print('Учитывать регистр в тексте и подстроке? Введите да или нет')
s = (input())

if s == 'нет':
    text.lower()
    substring.lower()
    
start_time = t.now()
print ('Подстрока найдена по индексу:', kmp_search(text, substring, 0))
end_time = t.now()
print('Duration: {}'.format(end_time - start_time))     

Введите текст: итак Smelly cat What are They feeding You такие дела
Введите подстроку для поиска: at ar
Учитывать регистр в тексте и подстроке? Введите да или нет
да
Подстрока найдена по индексу: 18
Duration: 0:00:00.000999


#### Упрощенный алгоритм Бойера-Мура

In [7]:
# формирование таблицы смещений
def offset_table(string):
    table = dict() 
    length = len(string)
    for i in range(length - 1):
        table[string[i]] = length - i - 1
    if not string[length - 1] in table:
        table[string[length - 1]] = length
    return table

def bm_search(string, substring):
    table = offset_table(substring)
    sub_length = len(substring)
    current_step = sub_length - 1 #счетчик проверяемого символа в строке

    while current_step < len(string):  #основной цикл для поиска подстроки
        s_sought_i = sub_length - 1 
        for i in range(current_step, current_step - sub_length, -1): #от конца к началу
            if string[i] != substring[s_sought_i]: #проверка последнего символа подстроки и символа в строке 
                if string[i] in table:
                    step = table[string[i]]
                else:
                    step = sub_length
                break

            if i == current_step - sub_length + 1:  #если конец
                return i
            s_sought_i -= 1

        current_step += step  #двигаем место поиска на шаг

text = input('Введите текст: ')
substring = input('Введите подстроку для поиска: ')
print('Учитывать регистр в тексте и подстроке? Введите да или нет')
s = (input())

if s == 'нет':
    text.lower()
    substring.lower()
    
start_time = t.now()
print ('Подстрока найдена по индексу:', bm_search(text, substring))
end_time = t.now()
print('Duration: {}'.format(end_time - start_time))    

Введите текст: итак Smelly cat What are They feeding You такие дела
Введите подстроку для поиска: hat
Учитывать регистр в тексте и подстроке? Введите да или нет
да
Подстрока найдена по индексу: 17
Duration: 0:00:00.000970


#### Стандартная функция поиска

In [10]:
text = input('Введите текст: ')
substring = input('Введите подстроку для поиска: ')
print('Учитывать регистр в тексте? Введите да или нет')
s = (input())

if s == 'нет':
    text.lower()
    substring.lower()
    
start_time = t.now()
print ('Подстрока найдена по индексу:', text.find(substring))
end_time = t.now()
print('Duration: {}'.format(end_time - start_time))

Введите текст: итак Smelly cat What are They feeding You такие дела sunny day silly song вот это да
Введите подстроку для поиска: ly so
Учитывать регистр в тексте? Введите да или нет
да
Подстрока найдена по индексу: 66
Duration: 0:00:00.000997


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [5]:
from queue import PriorityQueue

N = 4

#Функция, определяющая движение пятнашек
def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N) #возвращает пару частное-остаток
    offsets = []
    if i > 0: offsets.append(-N)     #сдвиг вниз
    if i < N - 1: offsets.append(N)  #сдвиг вверх
    if j > 0: offsets.append(-1)     #сдвиг вправо
    if j < N - 1: offsets.append(1)  #сдвиг влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x == blank else position[blank] if x == swap else position[x] for x in range(N*N))

#Функция, определяющая есть ли у пятнашек конечное решение
def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation: #перебор в перестановках
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:
    #Функция, принимающая позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    #Функция, срабатывающая при сравнении объекта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    #Функция, срабатывающая при использовании объекта как строки
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) #

for i in range(0, N*N, N))
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 13, 14, 11, 0]

#Проверяем, можно ли в данной расстановке найти решение
if parity(start) == 0:
    print('Решения нет!')
else:
    start = tuple(start)

    p = Position(start, 0)

    print(p)
    print()

    #Передаем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    #Кортеж посещенных позиций
    visited = set([p])

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        #Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        #Кладем в очередь все соседние позиции. Повторяем пока не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    #Последовательное решение пятнашек
    path = []
    #Конечная позиция
    prev = p
    #Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        #Запоминаем откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
  9 10 12 15
 13 14 11  0

[15, 12, 11, 15]


### Вывод

Выполняя лабораторную работу, мы познакомились с алгоритмом Кнута-Морриса-Пратта и прощенным алгоритмом Бойера-Мура, а затем реализовали их. Время поиска сравнили со стандартной функицей поиска в строке. Наиболее быстрым оказался агоритм Бойера-Мура. Также был реализован процесс игры в пятнашки.